In [49]:
import joblib
import os

from osgeo import gdal
from osgeo import osr

import helper
from feature_optimization import FeatureOptimizer
import feature_opt_functions as funcs
from indices import *

from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPClassifier

In [50]:
model_name = "nmdibased_11_6"
val_index = 0
model_index = 0

save_path = os.path.join(r"D:\SWIFTT_Chornobil\prediction", model_name)

if not os.path.exists(save_path):
    os.mkdir(save_path)

In [51]:
dictOfStressedImageNames = {}
for v in helper.getStressedImagesNames('stress_date.xlsx'):
    if v is not None:
        dictOfStressedImageNames[v[0]] = v[1]

dictOfReferenceImageNames = {}
for v in helper.getStressedImagesNames('reference_date.xlsx'):
    if v is not None:
        dictOfReferenceImageNames[v[0]] = v[1]

tilesPerGroup = None
with open(os.path.join("subdivs", "tiles_2img_2.json"), 'r') as fout:
    tilesPerGroup = json.load(fout)

train_tiles = tilesPerGroup[0]

train_data = helper.getH_and_S(dictOfStressedImageNames, train_tiles)
train_data_ref = helper.getH_and_S(dictOfReferenceImageNames, train_tiles)

(92, 4)
(92, 4)


In [52]:
encoder = IndicesClassEncoderEq([NMDIbased], list(range(1, 12)))

args = { 
    "num_generations":1, 
    "num_parents_mating":3,
    "parent_selection_type":"sss",
    "keep_elitism":1,
    "sol_per_pop":5,
    "mutation_probability":0.25,
    "parallel_processing":8
    }

opt = FeatureOptimizer(encoder, 16,
                        funcs.bhattacharyya_distance, 
                        funcs.spearman_independency, 
                        optimization_method="genetic",
                        optimizer_args=args,
                        informativeness_threshold=0.05, 
                        independency_threshold=0.05,
                        set_independency="geom_mean")

opt.fit(None, None, False, False)

#BANDS model
# opt.selected_features = list(range(11))
# seed = 1837588067
# insert = False

# NMDIbased 11+6 model
opt.selected_features = [255, 1080, 22, 127, 1102, 96]
seed = 25357148
insert = True

# FRAC3 16 model
# opt.selected_features = [515, 243, 13, 1163, 1315, 32, 608, 1191, 486, 1069, 697, 760, 172, 873, 752]
# seed = 1677770182
# insert = False

# HUE 16
# opt.selected_features = [856, 592, 1139, 1148, 505, 218, 164, 98, 144, 519, 119, 1300, 182, 896, 131]
# seed = 632434116
# insert = False



indices_train_H = helper.leaveFinite(opt.transform_series([train_data[0], train_data_ref[0]], insert)).swapaxes(0, 1)
indices_train_S = helper.leaveFinite(opt.transform_series([train_data[1], train_data_ref[1]], insert)).swapaxes(0, 1)

scaler = RobustScaler(unit_variance=True)

indices_train_H = scaler.fit_transform(indices_train_H)
indices_train_S = scaler.transform(indices_train_S)

train_X, train_y = helper.joinData(indices_train_H, indices_train_S)

model = MLPClassifier(hidden_layer_sizes=(16), max_iter=60, early_stopping=True, random_state=np.random.RandomState(seed))
model.fit(train_X, train_y)


MLPClassifier(early_stopping=True, hidden_layer_sizes=16, max_iter=60,
              random_state=RandomState(MT19937) at 0x1F8805DE740)

In [53]:
def leaveFinite(data):
    return data[:, (~np.isnan(data).any(axis=0)) & (np.isfinite(data).all(axis=0))]

def prepareData(data, converter):
    return leaveFinite(converter.convert(data)).swapaxes(0, 1)

chunks = os.listdir(r"D:\SWIFTT_Chornobil\summer")
for name in chunks:
    if str.endswith(name, ".xml"):
        continue

    print(f"Start {name}")

    img0 = gdal.Open(os.path.join(r"D:\SWIFTT_Chornobil\summer", name))
    img0_bands = helper.getBands(img0)

    img1 = gdal.Open(os.path.join(r"D:\SWIFTT_Chornobil\winter", name))
    img1_bands = helper.getBands(img1)

    print(img0_bands.shape)
    sub = img0_bands.reshape((img0_bands.shape[0], img0_bands.shape[1] * img0_bands.shape[2]))
    sub_ref = np.clip(img1_bands.reshape((img1_bands.shape[0], img1_bands.shape[1] * img1_bands.shape[2])) - 0.0125, 1 / 65536, 1000000000)

    data_transformed = opt.transform_series([sub, sub_ref], insert).swapaxes(0, 1)

    sub_conv = scaler.transform(data_transformed)
    print(sub_conv.shape)
    ids = np.where(~np.isnan(sub_conv).any(axis=1))
    sub_conv = sub_conv[ids[0], :]
    print(sub_conv.shape)

    if (sub_conv.shape[0] > 0):
        predict = model.predict_proba(sub_conv)[:, 1]
    else:
        predict = [0]

    result = np.zeros(img0_bands.shape[1]* img0_bands.shape[2])
    # result[ids[1], ids[2]] = predict
    result[ids[0]] = predict
    result = result.reshape((img0_bands.shape[1], img0_bands.shape[2]))

    driver = gdal.GetDriverByName('GTiff')

    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(img0.GetProjectionRef())
    proj = raster_srs.ExportToWkt()

    output = driver.Create(os.path.join(save_path, name), img0.RasterXSize, img0.RasterYSize, 1, gdal.GDT_Float32)
    output.SetProjection(proj)
    output.SetGeoTransform(img0.GetGeoTransform())
    output.GetRasterBand(1).WriteArray(result)
    output.FlushCache()
    output = None

    print("Finish")

Start france_32.tif
(12, 330, 566)
(186780, 36)
(186769, 36)
Finish
Start _3325390.0_6656850.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(452613, 36)
Finish
Start _3325390.0_6667090.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(775692, 36)
Finish
Start _3325390.0_6677330.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(651924, 36)
Finish
Start _3325390.0_6687570.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(17920, 36)
Finish
Start _3325390.0_6697810.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(0, 36)
Finish
Start _3325390.0_6708050.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(0, 36)
Finish
Start _3335630.0_6656850.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(1019260, 36)
Finish
Start _3335630.0_6667090.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(1017202, 36)
Finish
Start _3335630.0_6677330.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(1042220, 36)
Finish
Start _3335630.0_6687570.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(702222, 36)
Finish
Start _3335630.0_6697810.0_1024.tif
(12, 1024, 1024)
(1048576, 36)
(18